In [25]:
elastic_rods_dir = '../../../../elastic_rods/python/'
weaving_dir = '../../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, optimization_visualization_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
importlib.reload(optimization_visualization_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin,
                                   write_distance_to_linkage_mesh)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, contact_optimization, get_double_side_view, show_selected_joints, highlight_rod_and_joint)
import vis.fields
import matplotlib.cm as cm
import time
from optimization_visualization_helper import (compute_visualization_data_from_raw_data, get_objective_components_stage1, get_objective_components_stage2, get_objective_components_stage3, set_figure_label_and_limit, Visualization_Setting, plot_objective, plot_ribbon_component_analysis)

In [3]:
import parallelism
parallelism.set_max_num_tbb_threads(12)
parallelism.set_hessian_assembly_num_threads(4)
parallelism.set_gradient_assembly_num_threads(4)

In [4]:
rod_length = 0.21162395234664752
width = rod_length / 15 * 5
thickness = width / 5 * 0.35
print(width, thickness)
# width *= 0.5
# thickness *= 0.5

io = InputOrganizer('simple_hemoglobin_5_1', thickness, width, weaving_dir)
#  [ ear 214, ear 255, nose 39]

0.0705413174488825 0.004937892221421775


In [5]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 1
sw = 10
drw = 0.1
dsw = 0.1

In [6]:
import pickle 
import gzip
# pickle.dump(curved_linkage, gzip.open('hemoglobin_test_20201202.pkl.gz', 'w'))
# pickle.load(gzip.open('hemoglobin_three_stages_results/hemoglobin_contact_opt_2020_1201.pkl.gz', 'r'))

In [7]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewerWithSurface(curved_linkage, io.SURFACE_PATH)
set_surface_view_options(curved_linkage_view)
curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0), quaternion=(…

In [31]:
curved_linkage_view_2 = linkage_vis.LinkageViewer(curved_linkage)
curved_linkage_view_2.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0), quaternion=(…

In [8]:
centerline_pts = curved_linkage.centerLinePositions()

In [10]:
centerline_points = np.reshape(centerline_pts, (int(len(centerline_pts)/3), 3))

In [13]:
closest_clp = curved_linkage.get_closest_point_for_visualization(centerline_pts)

In [9]:
set_surface_view_options(curved_linkage_view, surface_transparent = True)

In [15]:
# Get rest length.
segment_rest_length = []
for i in range(curved_linkage.numSegments()):
    rod = curved_linkage.segment(i).rod
    pts = curved_linkage.get_closest_point_for_visualization(np.array(rod.deformedPoints()).flatten())
    pts = np.reshape(pts, (int(len(pts)/3), 3))
    rls = np.array([la.norm(pts[i+1] - pts[i]) for i in range(len(rod.deformedPoints())-1)])
    curved_linkage.segment(i).rod.setRestLengths(rls)
#     because the segment rest length only include from joint to joint (so half of the first and last edge length is not considered in the segment length)
    segment_rest_length.append(sum(rls) - 0.5 * (rls[0] + rls[-1]))
curved_linkage.setPerSegmentRestLength(segment_rest_length)
# Get rest kappa.
for i in range(curved_linkage.numSegments()):
    rod = curved_linkage.segment(i).rod
    pts = curved_linkage.get_closest_point_for_visualization(np.array(rod.deformedPoints()).flatten())
    pts = np.reshape(pts, (int(len(pts)/3), 3))

    normals = [curved_linkage.get_closest_point_normal(pt) for pt in pts]
    edges = np.array([pts[i+1] - pts[i] for i in range(len(pts)-1)])

    def get_rest_kappa(e1, e2, pn):
        cb = 2 * np.cross(e1, e2) / (la.norm(e1) * la.norm(e2) + np.dot(e1, e2))
        return np.dot(cb, pn)
    
    kappas = np.array([get_rest_kappa(edges[i], edges[i+1], normals[i+1]) for i in range(len(edges) - 1)])
    current_rk = rod.restKappas()

    for j in range(len(current_rk) - 2):
        current_rk[j+1][0] = kappas[j]
    curved_linkage.segment(i).rod.setRestKappas(current_rk)

In [16]:
seg = curved_linkage.segment(0)

In [17]:
df = rod.deformedConfiguration()

In [ ]:
df.

In [28]:
renderCam = np.load('../../../optimization_diagram_results/simple_hemoglobin_5_1/video/renderCam.npy')

In [29]:
def renderToFile(view, renderCam, path):
    orender = view.offscreenRenderer(width=2048, height=2048)
    orender.setCameraParams(renderCam)
    orender.render()
    orender.save(path)

In [33]:
distance_color = write_distance_to_linkage_mesh(curved_linkage, max(io.RIBBON_CS), None, return_distance_field = True)
curved_linkage_view_2.update(scalarField = distance_color[:, :3])
renderToFile(curved_linkage_view_2, renderCam, 'manual_hemoglobin.png')

In [19]:
curved_linkage.attraction_weight = 1e-5

In [20]:
def eqm_callback(prob, i):
    pass
#    curved_linkage_view.update()

In [21]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)
curved_linkage_view.update()

0	0.052202	14.4428	14.4428	1	1
1	0.0110946	0.996788	0.996788	1	1
2	0.0072062	0.451462	0.451462	1	1
3	0.00502932	0.249102	0.249102	1	1
4	0.00362972	0.14037	0.14037	1	1
5	0.00270922	0.0799015	0.0799015	1	1
6	0.00210743	0.0474745	0.0474745	1	1
7	0.00169819	0.0307748	0.0307748	1	1
8	0.00138506	0.0220975	0.0220975	1	1
9	0.00111215	0.0197174	0.0197174	1	1
10	0.000896418	0.0190456	0.0190456	1	1
11	0.000749794	0.00985147	0.00985147	1	1
12	0.000616594	0.00862693	0.00862693	1	1
13	0.000463506	0.0152646	0.0152646	1	1
14	0.000316093	0.023724	0.023724	1	1
15	0.000230635	0.0163629	0.0163629	1	1
16	0.000195686	0.0154379	0.0154379	0.5	0
17	0.000185158	0.0172567	0.0172567	1	0
18	0.000176555	0.0136527	0.0136527	0.5	1
19	0.000175652	0.0152409	0.0152409	0.5	0
20	0.000173697	0.0110695	0.0110695	1	0
21	0.000172025	0.00394789	0.00394789	1	0
22	0.00017132	0.00688247	0.00688247	1	0
23	0.000170782	0.00184023	0.00184023	1	0
24	0.000170702	0.00539617	0.00539617	1	0
25	0.000170469	0.000460785	0.000460785	1	0
26	0.

In [17]:
# Visualize the stresses on a single rod before and after freeing it from the weave.
rod_stress_view = optimization_visualization_helper.freedRibbonEquilibriumVisualization(curved_linkage, 0)
rod_stress_view.show()

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0), …

In [18]:
# optimizer = initialize_stage_2_optimizer(curved_linkage, io.SURFACE_PATH, curved_save_tgt_joint_pos, curved_linkage_view, rw, sw, inner_gradTol = 1e-7)

In [19]:
# start = time.time()
# optimizer, opt_iterateData, weight_change_iterations = stage_2_optimization(optimizer, curved_linkage, io.SURFACE_PATH, curved_save_tgt_joint_pos, curved_linkage_view, -1, -3, 3, return_weight_change_iteration = True)
# print(time.time() - start)

In [20]:
renderCam = curved_linkage_view.getCameraParams()
def renderToFile(path):
    orender = curved_linkage_view.offscreenRenderer(width=2048, height=2048)
    orender.setCameraParams(renderCam)
    orender.render()
    orender.save(path)

In [21]:
renderToFile('hemoglobin_stage2.png')

In [ ]:
opt_structure_view = get_structure_analysis_view(curved_linkage)
opt_structure_view.show()

In [ ]:
import linkage_optimization
cb = pipeline_helper.WeavingOptimizationCallback(optimizer, curved_linkage, curved_linkage_view)

optimizer.objective.terms[-1].term.weight = 1e6
optimizer.objective.terms[-1].term.normalWeight = 10
optimizer.objective.terms[-1].term.tangentialWeight = 1
optimizer.objective.terms[-1].term.normalActivationThreshold = -1e-5

In [ ]:
optimizer.objective.weightedValues()

In [ ]:
optimizer.WeavingOptimize(linkage_optimization.WeavingOptAlgorithm.NEWTON_CG, 2000, 1.0, 1e-2, cb, minRestLen=0.1)

In [ ]:
renderToFile('hemoglobin_stage3a.png')

In [ ]:
optimizer.objective.weightedValues()

In [ ]:
optimizer.objective.terms[-1].term.weight = 5e6
optimizer.objective.terms[-1].term.normalWeight = 10
optimizer.objective.terms[-1].term.tangentialWeight = 1
optimizer.objective.terms[-1].term.normalActivationThreshold = -1e-5

In [ ]:
optimizer.objective.weightedValues()

In [ ]:
optimizer.WeavingOptimize(linkage_optimization.WeavingOptAlgorithm.NEWTON_CG, 2000, 1.0, 1e-2, cb, minRestLen=0.1)

In [ ]:
renderToFile('hemoglobin_stage3b.png')

In [ ]:
optimizer.objective.weightedValues()

In [ ]:
# save_path = os.getcwd()
# write_all_output(curved_linkage, io.SUBDIVISION_RESOLUTION, io.MODEL_NAME, io.RIBBON_NAME, False, 100, min(io.RIBBON_CS), target_width = 5, flip_angles = False, width = max(io.RIBBON_CS))
# os.chdir(save_path)

In [ ]:
# curved_linkage.setExtendedDoFsPSRL(np.load('results/simple_hemoglobin_5_1/simple_hemoglobin_5_1_dof.npy'))

In [ ]:
# optimizer = initialize_stage_2_optimizer(curved_linkage, io.SURFACE_PATH, curved_save_tgt_joint_pos, curved_linkage_view, rw, sw)

In [ ]:
optimizer, contact_opt_iterateData = contact_optimization(optimizer, curved_linkage, curved_linkage_view, minRestLen=0.14566537396077858, contact_weight = 1e6, normalWeight = 5, torqueWeight = 2, maxIter=200)

In [ ]:
def set_weight_for_one_type(optimizer, curr_type):
    optimizer.beta = 0
    optimizer.gamma = 0
    optimizer.smoothing_weight = 0
    optimizer.rl_regularization_weight = 0
    optimizer.contact_force_weight = 0
    if curr_type == linkage_optimization.WeavingOptEnergyType.Full:
        optimizer.beta = 5e5
        optimizer.gamma = 1
        optimizer.smoothing_weight = sw
        optimizer.rl_regularization_weight = rw
        optimizer.contact_force_weight = 1e4
    elif curr_type == linkage_optimization.WeavingOptEnergyType.Target:
        optimizer.beta = 5e5
    elif curr_type == linkage_optimization.WeavingOptEnergyType.Smoothing:
        optimizer.smoothing_weight = sw
    elif curr_type == linkage_optimization.WeavingOptEnergyType.Regularization:
        optimizer.rl_regularization_weight = rw
    elif curr_type == linkage_optimization.WeavingOptEnergyType.Elastic:
        optimizer.gamma = 1
    elif curr_type == linkage_optimization.WeavingOptEnergyType.ContactForce:
        optimizer.contact_force_weight = 1e4

In [ ]:
import linkage_optimization
import fd_weaver_editor

In [ ]:
direction = np.random.uniform(0, 1, optimizer.numParams())

In [ ]:
set_weight_for_one_type(optimizer, linkage_optimization.WeavingOptEnergyType.Full)
fd_weaver_editor.gradient_convergence_plot(optimizer, direction, 'Full', linkage_optimization.WeavingOptEnergyType.Full)

In [ ]:
second_structure_view = get_structure_analysis_view(curved_linkage)
second_structure_view.show()

In [ ]:
# np.save('hemoglobin_self_intersection_2020_1126.npy', curved_linkage.getExtendedDoFsPSRL())

In [ ]:
# len(opt_iterateData)

In [ ]:
len(contact_opt_iterateData)

In [ ]:
# len(iterateData)

In [ ]:
# import pickle
# import gzip
# pickle.dump(curved_linkage, gzip.open('hemoglobin_contact_opt_2020_1201.pkl.gz', 'w'))

In [ ]:
# np.save('hemoglobin_contact_opt_data.npy', contact_opt_iterateData)

In [ ]:
contact_forces = [curr_iter['ContactForce'] for curr_iter in contact_opt_iterateData]

In [ ]:
elastic_energys = [curr_iter['ElasticEnergy'] for curr_iter in contact_opt_iterateData]

In [ ]:
target_fitting = [curr_iter['TargetFitting'] for curr_iter in contact_opt_iterateData]

In [ ]:
plt.plot(contact_forces)

In [ ]:
plt.plot(elastic_energys)

In [ ]:
plt.plot(target_fitting)

In [ ]:
# save_path = os.getcwd()
# write_all_output(curved_linkage, io.SUBDIVISION_RESOLUTION, io.MODEL_NAME + '_contact_opt', io.RIBBON_NAME + '_contact_opt', False, 100, min(io.RIBBON_CS), target_width = 5, flip_angles = False, width = max(io.RIBBON_CS))
# os.chdir(save_path)

In [ ]:
# curved_optimizer_energy = curved_linkage.energy()
# validation_curved_linkage = optimizer.getLinesearchWeaverLinkage()
# validation_curved_linkage.attraction_weight = 1e-7
# with so(): elastic_rods.compute_equilibrium(validation_curved_linkage, options = OPTS, fixedVars = [])
# validation_curved_view = linkage_vis.LinkageViewer(validation_curved_linkage, width=1024, height=640)
# validation_curved_energy = validation_curved_linkage.energy()
# print(abs((validation_curved_energy-curved_optimizer_energy)/curved_optimizer_energy))

### Energy Plots

In [ ]:
DEBUG = True
def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)

vs = Visualization_Setting()

name = io.MODEL_NAME
data_filename = '{}/{}_{}_data.npy'.format('results/{}'.format(name), name, 'full_data' if not DEBUG else 'finite_sample')
optimization_data_filename = '{}/{}_optimization.npy'.format('results/{}'.format(name), name)
print('Computing Visualization Data ', name)

iterateData, opt_iterateData = np.load(optimization_data_filename, allow_pickle = True)
contact_iterateData = np.load('hemoglobin_three_stages_results/hemoglobin_contact_opt_data.npy', allow_pickle = True)

In [ ]:
sum([curr_iter['iteration_time'] for curr_iter in iterateData]), sum([curr_iter['iteration_time'] for curr_iter in opt_iterateData]), sum([curr_iter['iteration_time'] for curr_iter in contact_opt_iterateData][1:])

In [ ]:

name = io.MODEL_NAME
data_filename = '{}/{}_{}_data.npy'.format('results/{}'.format(name), name, 'full_data' if not DEBUG else 'finite_sample')
optimization_data_filename = '{}/{}_optimization.npy'.format('results/{}'.format(name), name)
print('Computing Visualization Data ', name)
# np.save(optimization_data_filename, [iterateData, opt_iterateData])

dps_total_absolute_curvature, dps_total_ribbon_length, dps_distance_to_surface, dps_distance_to_joint, dps_eqm_dofs, dps_elastic_energy, dps_separation_force, dps_tangential_force = compute_visualization_data_from_raw_data(iterateData, io, DEBUG, is_bunny = False)

opt_total_absolute_curvature, opt_total_ribbon_length, opt_distance_to_surface, opt_distance_to_joint, opt_eqm_dofs, opt_elastic_energy, opt_separation_force, opt_tangential_force  = compute_visualization_data_from_raw_data(opt_iterateData, io, DEBUG, is_bunny = False)

contact_total_absolute_curvature, contact_total_ribbon_length, contact_distance_to_surface, contact_distance_to_joint, contact_eqm_dofs, contact_elastic_energy, contact_separation_force, contact_tangential_force  = compute_visualization_data_from_raw_data(contact_iterateData, io, DEBUG, is_bunny = False)


combined_energy = np.array(dps_elastic_energy + opt_elastic_energy + contact_opt_elastic_energy )
combined_rest_length = np.array(dps_total_ribbon_length + opt_total_ribbon_length + contact_opt_total_ribbon_length)
combined_total_absolute_curvature = np.array(dps_total_absolute_curvature + opt_total_absolute_curvature + contact_opt_total_absolute_curvature)
combined_distance_to_surface = np.array(dps_distance_to_surface + opt_distance_to_surface + contact_opt_distance_to_surface)
combined_dis_to_target_joint = np.array(dps_distance_to_joint + opt_distance_to_joint + contact_opt_distance_to_joint)
combined_separation_force = np.array(dps_separation_force + opt_separation_force + contact_separation_force)
combined_tangential_force = np.array(dps_tangential_force + opt_tangential_force + contact_tangential_force)


curr_data = np.array([combined_energy, combined_rest_length, combined_total_absolute_curvature, combined_distance_to_surface, combined_dis_to_target_joint, dps_eqm_dofs, opt_eqm_dofs])
# np.save(data_filename, curr_data)

dps_objective_elastic, dps_objective_smooth, dps_objective_length, dps_total_objective = get_objective_components_stage1(iterateData)
opt_objective_elastic, opt_objective_target, opt_objective_length, opt_objective_smooth, opt_total_objective = get_objective_components_stage2(opt_iterateData)
contact_objective_elastic, contact_objective_target, contact_objective_length, contact_objective_smooth, contact_total_objective, contact_contact_force = get_objective_components_stage3(contact_opt_iterateData)

combined_weighted_smoothing = np.concatenate((dps_objective_smooth, opt_objective_smooth, contact_objective_smooth), axis = None)

# # Objective plots
# plot_objective(vs, iterateData, dps_total_objective, '{}/{}_stage_1_objective.png'.format('results/{}'.format(name), name), vs.stage_1_label)
# plot_objective(vs, opt_iterateData, opt_total_objective, '{}/{}_stage_2_objective.png'.format('results/{}'.format(name), name), vs.stage_2_label)
# plot_objective(vs, contact_opt_iterateData, contact_total_objective, '{}/{}_stage_3_objective.png'.format('results/{}'.format(name), name), vs.stage_3_label)





In [ ]:
plot_ribbon_component_analysis(iterateData, dps_elastic_energy, dps_total_ribbon_length, dps_objective_smooth, dps_total_absolute_curvature, dps_distance_to_surface, dps_separation_force, dps_tangential_force, DEBUG, combined_energy, combined_rest_length, combined_weighted_smoothing, combined_total_absolute_curvature, combined_distance_to_surface, combined_separation_force, combined_tangential_force, vs, name, 1, vs.stage_1_label)

In [ ]:
plot_ribbon_component_analysis(opt_iterateData, opt_elastic_energy, opt_total_ribbon_length, opt_objective_smooth, opt_total_absolute_curvature, opt_distance_to_surface, opt_separation_force, opt_tangential_force, DEBUG, combined_energy, combined_rest_length, combined_weighted_smoothing, combined_total_absolute_curvature, combined_distance_to_surface, combined_separation_force, combined_tangential_force, vs, name, 2, vs.stage_2_label)

In [ ]:
plot_ribbon_component_analysis(iterateData, contact_elastic_energy, contact_total_ribbon_length, contact_objective_smooth, contact_total_absolute_curvature, contact_distance_to_surface, contact_separation_force, contact_tangential_force, DEBUG, combined_energy, combined_rest_length, combined_weighted_smoothing, combined_total_absolute_curvature, combined_distance_to_surface, combined_separation_force, combined_tangential_force, vs, name, 3, vs.stage_3_label)